In [1]:
import torch
import importlib
import wrapped.Dataloader as Dataloader
importlib.reload(Dataloader)

from wrapped.Dataloader import prep_data_local

In [2]:
path = "./Sen2_MTC_Mini"
patch_size = 128
stride = 128
batch_size = 16
train_ratio = .7
val_ratio = .15

train_loader, val_loader, test_loader = prep_data_local(path=path, 
                                                        patch_size=patch_size, 
                                                        stride=stride,
                                                        batch_size=batch_size,
                                                        train_ratio=train_ratio,
                                                        val_ratio=val_ratio)
#expect ~40s run on full Sen2_MTC data

[Sen2-MTC Data loaded] Total samples: 4200
Global mean: tensor([1888.0425, 1985.3403, 1909.0002, 3378.6661], dtype=torch.float64)
Global std: tensor([2090.9713, 2116.0823, 2255.2750, 1857.1011], dtype=torch.float64)
[Sen2-MTC Data loaded] Total samples: 4200


In [28]:
for batch in train_loader:
    print(batch['cloudy_seq'].shape)
    print(batch['clean'].shape)
    x = batch['cloudy_seq'][0]
    print(x.mean(), x.std())
    break

torch.Size([16, 3, 4, 128, 128])
torch.Size([16, 4, 128, 128])
tensor(-0.0653) tensor(0.5254)


In [7]:
import wrapped.model as model
importlib.reload(model)
import wrapped.utils as utils
importlib.reload(utils)
from wrapped.model import build_forward_model, backward_sampler

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_dict = build_forward_model(
    in_channels = 4,
    base_channels = 32,
    num_stages = 2,
    latent_dim = 128,
    T_cloud=3,
    T_diffusion=750,
    device=device
)
base_encoder = model_dict['base_encoder']
cloud_encoder = model_dict['cloud_encoder']
forwarder = model_dict['forwarder']
denoiser = model_dict['denoiser']

import wrapped.utils as utils
cloud_enc_count = utils.count_params(cloud_encoder)
forward_enc_count = utils.count_params(forwarder)
denoiser_count = utils.count_params(denoiser)
print(f"Total number of parameters: {cloud_enc_count+forward_enc_count+denoiser_count}")

Total number of parameters: 4292549


In [9]:
import wrapped.trainers as trainers
importlib.reload(trainers)
from wrapped.trainers import forward_trainer

epochs = 5
params = list(cloud_encoder.parameters()) + list(denoiser.parameters())
optimizer = torch.optim.AdamW(
    [
        {"params": denoiser.parameters(),      "lr": 1e-4},
        {"params": cloud_encoder.parameters(), "lr": 5e-5},
    ],
    weight_decay=1e-6,
)
forward_trainer(epochs=epochs,
                train_loader=train_loader,
                optimizer=optimizer,
                forwarder=forwarder,
                cloud_encoder=cloud_encoder,
                denoiser=denoiser,
                device=device)

NameError: name 'train_loader' is not defined

In [10]:
cloud_encoder.eval()
denoiser.eval()
batch = next(iter(train_loader))
cloudy_seq = batch['cloudy_seq'].to(device)

x0 = backward_sampler(cloudy_seq=cloudy_seq,
                      cloud_encoder=cloud_encoder,
                      denoiser=denoiser,
                      forwarder=forwarder,
                      num_steps=750)

NameError: name 'train_loader' is not defined

In [11]:
from wrapped.utils import visualize
visualize(cloudy_seq=cloudy_seq,
          batch=batch,
          x0=x0)

NameError: name 'cloudy_seq' is not defined